In [3]:
class ToDo:

    progress = 0  # class-level variable

    def __init__(self, description, status):
        self.description = description
        self.is_done = status

    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            ToDo.progress = ToDo.progress + 1
        self.is_done = True
        
    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            ToDo.progress = ToDo.progress - 1
        self.is_done = False
        
class Deadline(ToDo):
    
    progress = 0
    
    def __init__(self, description, status, deadline):
        self.deadline = deadline
        ToDo.__init__(self, description, status)
        
    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            Deadline.progress = Deadline.progress + 1
        self.is_done = True
        
    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            Deadline.progress = Deadline.progress - 1
        self.is_done = False
        
        
items = []
items.append(ToDo("Read book", False))
items.append(ToDo("borrow book", False))
items.append(Deadline("Make coffee", False, "Monday"))

def get_current_progress():
    status = {'Todo': 0, 'Deadline': 0}
    tFlag = 0
    dFlag = 0
    
    for obj in items:
        if isinstance(obj, Deadline) and dFlag == 0:
            print("added to deadline count\n")
            status['Deadline'] = obj.progress
            dFlag = 1
        elif tFlag == 0:
            status['Todo'] = obj.progress
            tFlag = 1
    print(">>> Progress for this session: todos {} deadlines {}".format(status['Todo'], status['Deadline']))

items[0].mark_as_done()
get_current_progress()
items[2].mark_as_done()
get_current_progress()
items[2].mark_as_pending()
get_current_progress()

added to deadline count

>>> Progress for this session: todos 1 deadlines 0
added to deadline count

>>> Progress for this session: todos 1 deadlines 1
added to deadline count

>>> Progress for this session: todos 1 deadlines 0


In [37]:
class ToDo:
    
    progress = 0 # class-level variable
    TYPE_KEY = 'T'

    def __init__(self, description, status):
        self.description = description
        self.is_done = status
        
    def __str__(self):        
        return self.__status_as_icon().center(6) + ' ' + '| ' + ' '.center(5) \
    + '| ' + self.description.ljust(27) + ' ' + '| ' + '-'

    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            ToDo.progress = ToDo.progress + 1
        self.is_done = True

    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            if ToDo.progress != 0:
                ToDo.progress = ToDo.progress - 1
        self.is_done = False

    def __status_as_icon(self):
        return 'X' if self.is_done else '-'
    
    def as_csv(self):
        """ Return the details of todo object as a list,
        suitable to be stored in a csv file.
        """
        return [self.TYPE_KEY, self.description, 'done' if self.is_done else 'pending']
    
    def as_item(self):
        """ Return the name of the item as a string."""
        return self.description
    
class Deadline (ToDo):
    
    progress = 0 # class-level variable
    TYPE_KEY = 'D'

    def __init__(self, description, status, by):
        super().__init__(description, status)
        self.by = by        
    
    def __str__(self):
        s = super().__str__()
        return s[:-1] + self.by
        
    def as_csv(self):
        c = super().as_csv()
        c.append(self.by)
        return c
        
    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            Deadline.progress = Deadline.progress + 1
        self.is_done = True
        
    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            if Deadline.progress != 0:
                Deadline.progress = Deadline.progress - 1
        self.is_done = False

In [54]:
import datetime
from tkinter import *

import csv
import copy
from io import StringIO

import sys

class GUI:

    def __init__(self, task_manager):
        self.task_manager = task_manager
        self.window = Tk()
        self.window.geometry('800x700')  # set Window size
        self.window.title('Monty')  # set Window title

        self.input_box = Entry(self.window)  # create an input box
        self.input_box.pack(padx=5, pady=5, fill='x')  # make the input box fill the width of the Window
        self.input_box.bind('<Return>', self.command_entered)  # bind the command_entered function to the Enter key
        self.input_box.focus()  # set focus to the input box

        # add a text area to show the chat history
        self.history_area = Text(self.window, width="50")
        self.history_area.pack(padx=5, pady=5, side=LEFT, fill="y")
        self.output_font = ('Courier New', 12)
        self.history_area.tag_configure('error_format', foreground='red', font=self.output_font)
        self.history_area.tag_configure('success_format', foreground='green', font=self.output_font)
        self.history_area.tag_configure('normal_format', font=self.output_font)

        # add a text area to show the list of tasks
        self.list_area = Text(self.window)
        self.list_area.pack(padx=5, pady=5, side=RIGHT, fill="both")
        self.list_area.tag_configure('normal_format',  font=self.output_font)
        self.list_area.tag_configure('pending_format', foreground='red', font=self.output_font)
        self.list_area.tag_configure('done_format', foreground='green', font=self.output_font)

        # show the welcome message and the list of tasks
        self.update_chat_history('start', 'Welcome to Monty!', 'success_format')
        self.update_task_list(self.task_manager.items)

    def update_chat_history(self, command, response, status_format):
        """
        status_format: indicates which color to use for the status message
          can be 'error_format', 'success_format', or 'normal_format'
        """
        current_time = datetime.datetime.now().strftime("%H:%M:%S")
        self.history_area.insert(1.0, '-' * 40 + '\n', 'normal_format')
        self.history_area.insert(1.0, '>>> ' + response + '\n', status_format)
        self.history_area.insert(1.0, 'You said: ' + command + '\n', 'normal_format')
        self.history_area.insert(1.0, current_time + '\n', 'normal_format')

    def update_task_list(self, tasks):
        self.list_area.delete('1.0', END)  # clear the list area
        for i, task in enumerate(tasks):
            if task.is_done:
                icon = 'X'
                output_format = 'done_format'
            else:
                icon = '-'
                output_format = 'pending_format'
            self.list_area.insert(END, icon + ' ' + str(i+1) + ' ' + task.description + '\n', output_format)

    def clear_input_box(self):
        self.input_box.delete(0, END)

    def command_entered(self, event):
        command = None
        try:
            command = self.input_box.get()
            if command.strip().lower() == 'exit':
                sys.exit()
            output = self.task_manager.execute_command(command)
            self.update_chat_history(command, output, 'success_format')
            self.update_task_list(self.task_manager.items)
            self.clear_input_box()
        except Exception as e:
            self.update_chat_history(command, str(e) + '\n' + self.task_manager.get_help(), 'error_format')

    def start(self):
        self.window.mainloop()


class TaskManager:
    
    items = []
    filename = 'monty7.csv'
    
    def load_data(self):
        TaskManager.__create_file_if_missing(self)
        with open(self.filename, 'r') as csvfile:
            file_handler = csv.reader(csvfile)
            for row in file_handler:
                if not row:
                    continue
                print(row)
                TaskManager.load_item_from_csv_line(row)
                print(TaskManager.items)
            return
        
    def __init__(self):
        self.load_data()
        self.items = TaskManager.items
        
    def __create_file_if_missing(self):
        open(self.filename, 'a').close()
        
    def load_item_from_csv_line(row):
        if row[0] == 'T':
            TaskManager.items.append(ToDo(row[1], True if row[2] == 'True' else False))
        elif row[0] == 'D':
            TaskManager.items.append(Deadline(row[1], True if row[2] == 'True' else False, row[3]))
        return
        
    def get_help(self):
        return 'help:\n...'

    def execute_command(self, command):
        if command == 'help':
            return self.get_help()
        elif command.startswith('add '):
            self.items.append([command[4:], False])
            return 'task added'
        else:
            raise Exception('Command not recognized')


GUI(TaskManager()).start()

['T', 'borrow book', 'True']
['T', 'read book', 'False']
[<__main__.ToDo object at 0x000001A1C2AD5640>, <__main__.ToDo object at 0x000001A1C33CD6D0>]
['D', 'watch movie', 'False', 'Monday']
[<__main__.ToDo object at 0x000001A1C2AD5640>, <__main__.ToDo object at 0x000001A1C33CD6D0>, <__main__.Deadline object at 0x000001A1C33F51C0>]
